In [ ]:
import akshare as ak
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import plotly.offline as py
from tqdm import tqdm

pd.options.plotting.backend = "plotly"

stocks = ak.stock_board_industry_cons_ths(symbol="白酒")

all_pe = pd.DataFrame()
all_pb = pd.DataFrame()
all_ps = pd.DataFrame()
all_dv = pd.DataFrame()
all_mv = pd.DataFrame()

for i in tqdm(range(len(stocks))):
    code = stocks.loc[i, "代码"]

    ind = ak.stock_a_lg_indicator(code)
    ind.to_csv("value-{}.csv".format(code))
    pe = pd.Series(list(ind["pe_ttm"]), index=pd.to_datetime(ind["trade_date"]))
    pb = pd.Series(list(ind["pb"]), index=pd.to_datetime(ind["trade_date"]))
    ps = pd.Series(list(ind["ps_ttm"]), index=pd.to_datetime(ind["trade_date"]))
    dv = pd.Series(list(ind["dv_ttm"]), index=pd.to_datetime(ind["trade_date"]))
    mv = pd.Series(list(ind["total_mv"]), index=pd.to_datetime(ind["trade_date"]))
    pe_df = pd.DataFrame({code: pe}).fillna(0)
    pb_df = pd.DataFrame({code: pb}).fillna(0)
    ps_df = pd.DataFrame({code: ps}).fillna(0)
    dv_df = pd.DataFrame({code: dv}).fillna(0)
    mv_df = pd.DataFrame({code: mv}).fillna(0)
    all_pe = pd.concat([all_pe, pe_df], axis=1)
    all_pb = pd.concat([all_pb, pb_df], axis=1)
    all_ps = pd.concat([all_ps, ps_df], axis=1)
    all_dv = pd.concat([all_dv, dv_df], axis=1)
    all_mv = pd.concat([all_mv, mv_df], axis=1)

cols = all_pe.columns.values
trace_list = []
for i in range(0, len(cols)):
    trace_list.append(go.Scatter(x=all_pe.index, y=all_pe[cols[i]], mode="lines", name=cols[i]))

fig = go.Figure(trace_list)
py.plot(fig)
# print(all_pe)
# print(all_pe[:])
# print(ind_df)
# plt.style.use('ggplot')
# color_palette = sns.color_palette("hls", 5)
# plt.rcParams['font.sans-serif'] = ['Times New Roman']
# fig = plt.figure(figsize=(16, 12))
# pd.set_option('plotting.backend', 'matplotlib')

# fig = make_subplots(rows=5, cols=1)

# fig.append_trace(go.Scatter(x=all_pe.index, y=all_pe[:], mode="lines", name="pe"), 1, 1)
# fig.update_layout(width=1500, height=(len(cols) * 600))
# fig.show()

In [ ]:
print(all_pe)

In [ ]:
cols = all_pe.columns.values
trace_list = []
for i in range(0, len(cols)):
    trace_list.append(go.Scatter(x=all_pe.index, y=all_pe[cols[i]], mode="lines", name=cols[i]))

fig = go.Figure(trace_list)
py.plot(fig)

# print(10 **8)

In [ ]:
import pandas as pd
import pandas_bokeh
import numpy as np

np.random.seed(42)
df = pd.DataFrame({"谷歌": np.random.randn(1000) + 0.2, 
                    "苹果": np.random.randn(1000) + 0.17}, index=pd.date_range('1/1/2022', periods=1000))

df = df.cumsum()
df = df + 50
pd.set_option('plotting.backend', '')

# pandas_bokeh.output_notebook()
df.plot_bokeh.line()

In [ ]:
#价值分析1: PE分析

import numpy as np
import pandas as pd
import akshare as ak
from tqdm import tqdm
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

def get_industry_pe(industry: str):
    stocks = ak.stock_board_industry_cons_ths(symbol=industry)

    all_stock = pd.DataFrame()

    stock_len = len(stocks)
    # 生成excel表格
    # sheet_writer = pd.ExcelWriter(f'{industry}_pe.xlsx')

    for i in tqdm(range(stock_len)):
        # market = 'sz'
        code = stocks.loc[i, "代码"]
        name = stocks.loc[i, "名称"]
        if (code.startswith('4') or code.startswith('8')):
            continue        
        
        lg_ind = ak.stock_a_lg_indicator(symbol=code)
        lg_ind = lg_ind[lg_ind['pe_ttm'].notnull()]
        lg_ind['code'] = code
        lg_ind['net_profit'] = lg_ind['total_mv'] / lg_ind['pe_ttm']
        all_stock = pd.concat([all_stock, lg_ind], ignore_index=True)

    all_stock.to_csv(f"stock_list_{industry}.csv", encoding='utf_8_sig')
    out = all_stock.groupby('trade_date')[['total_mv', 'net_profit']].sum()
    out['stock_size'] = all_stock.groupby('trade_date').size()
    out['average_pe'] = out['total_mv'] / out['net_profit']
    first_valid_item = out[out.stock_size >= 10].iloc[:1].index.to_numpy()
    # out.to_csv(f"{industry}_nodrop.csv", encoding='utf_8_sig')
    out = out.loc[out.index >= first_valid_item[0]]
    out.index = pd.to_datetime(out.index)
    # print(out.index)
    # out.to_csv(f"{industry}.csv", encoding='utf_8_sig')
    
    return out

def get_all_industry():
    all_industry = ak.stock_board_industry_summary_ths()

    return all_industry    

def get_all_industry_pe(industry_list:list):
    all_industry_pe = {}

    for industry in industry_list:
        print(f"get {industry}")
        
        industry_pe = get_industry_pe(industry)
        all_industry_pe[industry] = industry_pe    

    return all_industry_pe

def drawPe(all_industry_pe:dict):
    pd.options.plotting.backend = "matplotlib"

    # print(mpl.get_cachedir())
    plt.style.use('ggplot')
    color_palette = sns.color_palette("hls", len(all_industry_pe.keys()))
    plt.rcParams["font.sans-serif"] = ["SimHei"]
    plt.rcParams['axes.unicode_minus'] = False

    fig = plt.figure(figsize=(26, 20))

    index = 0
    for k,v in all_industry_pe.items():
        industry = k
        plt.subplot(len(all_industry_pe.keys()), 1, index+1)
        all_industry_pe[industry]['average_pe'].plot(kind='line', figsize=(26, 20), color=color_palette[index], label=industry)
        plt.xlabel('')
        plt.ylabel('pe ttm')
        plt.legend()
        index += 1

    plt.tight_layout()    

In [ ]:
all_industry = get_all_industry()

all_industry["板块"][:8].to_list()

In [ ]:
# all_industry_pe = get_all_industry_pe(['半导体及元件', '小家电','汽车服务','中药'])
all_industry_pe = get_all_industry_pe(all_industry["板块"][:8].to_list())

In [ ]:
drawPe(all_industry_pe)

In [ ]:
all_industry_pe['半导体及元件']['2022']

In [ ]:
import matplotlib as mpl
import matplotlib.pyplot as plt

def plot_demo():
    #print(mpl.get_cachedir())
    # 绘制折线图
    plt.rcParams["font.sans-serif"] = ["SimHei"]  # 设置字体
    plt.rcParams["axes.unicode_minus"] = False  # 正常显示负号
    year = [2017, 2018, 2019, 2020]
    people = [20, 40, 60, 70]
    # 生成图表
    plt.plot(year, people)
    plt.xlabel('年份')
    plt.ylabel('人口')
    plt.title('人口增长')
    # 设置纵坐标刻度
    plt.yticks([0, 20, 40, 60, 80])
    # 设置填充选项：参数分别对应横坐标，纵坐标，纵坐标填充起始值，填充颜色
    plt.fill_between(year, people, 20, color='green')
    # 显示图表
    # plt.savefig("./plt.png")
    plt.show()

plot_demo()